## 软件包加载

In [1]:
import numpy as np                     # 引入基础软件包numpy
import pandas as pd                    # 引入基础软件包pandas
from collections import OrderedDict    # OrderedDict用于记录模型的specification(声明) 
import pylogit as pl                   # 引入基础软件包logit模型软件包pylogit
import matplotlib.pyplot as plt        # 引入绘图软件包

## 数据读入

In [2]:
# 数据读入
data_path = u'../Chapters1_Data/long_data.csv'
raw_data = pd.read_table(data_path, sep=',', header=0)
raw_data.head(5)

C:\Users\TBBack\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,OBS_ID,ALT_ID,MODE,HINC,PSIZE,TTME,INVC,INVT,GC
0,1,0,0,35,1,69,59,100,70
1,1,1,0,35,1,34,31,372,71
2,1,2,0,35,1,35,25,417,70
3,1,3,1,35,1,0,10,180,30
4,2,0,0,30,2,64,58,68,68


## 模型格式声明

In [3]:
model_data = raw_data[['OBS_ID','ALT_ID','MODE','HINC','PSIZE','TTME','INVC','INVT']]

## 模型搭建

In [4]:
# NOTE: - 规范和变量名是有序字典。
#       - 键应该是长格式数据帧中的变量。唯一的例外是截距键“intercept”。
#       - 对于规范字典，值应该是整数列表。
#       - 在一个列表中，或者在最内部的列表中，整数表示的备选项的ID。
#       - 列表表示将共享系数的备选项。
basic_specification = OrderedDict()
basic_names = OrderedDict()

# 注意截距项包含选项个数减1个
basic_specification["intercept"] = [0, 1, 2]
basic_names["intercept"] = ['ASC_air', 'ASC_train', 'ASC_bus']
# 备选项属性的影响方式可以灵活指定
basic_specification["TTME"] = [[0, 1, 2]]
basic_names["TTME"] = ['TTME']
basic_specification["INVC"] = [[0, 1, 2, 3]]
basic_names["INVC"] = ['INVC']
basic_specification["INVT"] = [[0, 1, 2, 3]]
basic_names["INVT"] = ['INVT']

# 决策者的影响方式也可以灵活指定，但需要注意的是，由于每个选项的决策者属性都一样，因此保证可估计性，只对部分选项生效
basic_specification["HINC"] = [0, 1, 2]
basic_names["HINC"] = ['HINC_air', 'HINC_train', 'HINC_bus']
basic_specification["PSIZE"] = [0, 1, 2]
basic_names["PSIZE"] = ['PSIZE_air', 'PSIZE_train', 'PSIZE_bus']

In [5]:
# 模型创建
mnl = pl.create_choice_model(data = model_data,
                    alt_id_col="ALT_ID",
                    obs_id_col="OBS_ID",
                    choice_col="MODE",
                    specification=basic_specification,
                    model_type = "MNL",
                    names=basic_names)

C:\Users\TBBack\Anaconda3\lib\site-packages\pylogit\choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  design_matrix = np.hstack((x[:, None] for x in independent_vars))


In [6]:
# 模型估计&模型结果
mnl.fit_mle(np.zeros(12)) # 需要输入模型参数数量，根据之前的模型表达式即可得到
mnl.get_statsmodels_summary()
# | -------------------------------------------------------------
# |               coef     std.err z       P>|z|   [0.025  0.975]
# | -------------------------------------------------------------
# | ASC_air       6.0352   1.138   5.302   0.000   3.804   8.266
# | ASC_train     5.5735   0.711   7.836   0.000   4.179   6.968
# | ASC_bus       4.5047   0.796   5.661   0.000   2.945   6.064
# | TTME         -0.1012   0.011  -9.081   0.000  -0.123  -0.079
# | INVC         -0.0087   0.008  -1.101   0.271  -0.024   0.007
# | INVT         -0.0041   0.001  -4.627   0.000  -0.006  -0.002
# | HINC_air      0.0075   0.013   0.567   0.571  -0.018   0.033
# | HINC_train   -0.0592   0.015  -3.977   0.000  -0.088  -0.03
# | HINC_bus     -0.0209   0.016  -1.278   0.201  -0.053   0.011
# | PSIZE_air    -0.9224   0.259  -3.568   0.000  -1.429  -0.416
# | PSIZE_train   0.2163   0.234   0.926   0.355  -0.242   0.674
# | PSIZE_bus    -0.1479   0.343  -0.432   0.666  -0.820   0.524

Log-likelihood at zero: -291.1218
Initial Log-likelihood: -291.1218


C:\Users\TBBack\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:506: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.10 seconds.
Final log-likelihood: -172.4680


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                        MODE   No. Observations:                  210
Model:             Multinomial Logit Model   Df Residuals:                      198
Method:                                MLE   Df Model:                           12
Date:                     Fri, 07 Feb 2020   Pseudo R-squ.:                   0.408
Time:                             17:02:59   Pseudo R-bar-squ.:               0.366
AIC:                               368.936   Log-Likelihood:               -172.468
BIC:                               409.101   LL-Null:                      -291.122
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ASC_air         6.0352      1.138      5.302      0.000       3.804       8.266
ASC_train       5.5735      0.711      7.836      0.000       4.179       6.968
ASC_bus         4.5047      0.796      5.661      0.000       2.945       6.064
TTME           -0.1012      0.011     -9.081      0.000      -0.123      -0.079
INVC           -0.0087      0.008     -1.101      0.271      -0.024       0.007
INVT           -0.0041      0.001     -4.627      0.000      -0.006      -0.002
HINC_air        0.0075      0.013      0.567      0.571      -0.018       0.033
HINC_train     -0.0592      0.015     -3.977      0.000      -0.088      -0.030
HINC_bus       -0.0209      0.016     -1.278      0.201      -0.053       0.011
PSIZE_air      -0.9224      0.259     -3.568      0.000      -1.429      -0.416
PSIZE_train     0.2163      0.234      0.926      0.355      -0.242       0.674
PSIZE_bus      -0.1479      0.343     -0.432      0.666      -0.820       0.524
===============================================================================
"""

In [7]:
# NOTE: - 规范和变量名是有序字典。
#       - 键应该是长格式数据帧中的变量。唯一的例外是截距键“intercept”。
#       - 对于规范字典，值应该是整数列表。
#       - 在一个列表中，或者在最内部的列表中，整数表示的备选项的ID。
#       - 列表表示将共享系数的备选项。
basic_specification = OrderedDict()
basic_names = OrderedDict()

# 注意截距项包含选项个数减1个
basic_specification["intercept"] = [0, 1, 2]
basic_names["intercept"] = ['ASC_air', 'ASC_train', 'ASC_bus']
# 备选项属性的影响方式可以灵活指定
basic_specification["TTME"] = [[0, 1, 2]]
basic_names["TTME"] = ['TTME']
basic_specification["INVT"] = [[0, 1, 2, 3]]
basic_names["INVT"] = ['INVT']
# 决策者的影响方式也可以灵活指定，但需要注意的是，由于每个选项的决策者属性都一样，因此保证可估计性，只对部分选项生效
basic_specification["HINC"] = [[1, 2]]
basic_names["HINC"] = [ 'HINC_train_bus']
basic_specification["PSIZE"] = [0]
basic_names["PSIZE"] = ['PSIZE_air']

# 模型创建
mnl = pl.create_choice_model(data = model_data,
                    alt_id_col="ALT_ID",
                    obs_id_col="OBS_ID",
                    choice_col="MODE",
                    specification=basic_specification,
                    model_type = "MNL",
                    names=basic_names)

# 模型估计&模型结果
mnl.fit_mle(np.zeros(7))
mnl.get_statsmodels_summary()

# | -----------------------------------------------------------------
# |                   coef     std.err z       P>|z|   [0.025  0.975]
# | -----------------------------------------------------------------
# | ASC_air          5.6860   0.937   6.068   0.000   3.849   7.523
# | ASC_train        5.4034   0.603   8.959   0.000   4.221   6.585
# | ASC_bus          5.0128   0.623   8.051   0.000   3.792   6.233
# | TTME            -0.0992   0.011  -9.428   0.000  -0.12   -0.079
# | INVT            -0.0039   0.001  -4.489   0.000  -0.006  -0.002
# | HINC_train_bus  -0.0500   0.011  -4.484   0.000  -0.072  -0.028
# | PSIZE_air       -0.8997   0.245  -3.680   0.000  -1.379  -0.420
# |==================================================================

C:\Users\TBBack\Anaconda3\lib\site-packages\pylogit\choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  design_matrix = np.hstack((x[:, None] for x in independent_vars))


Log-likelihood at zero: -291.1218
Initial Log-likelihood: -291.1218


C:\Users\TBBack\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:506: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.03 seconds.
Final log-likelihood: -176.3203


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                        MODE   No. Observations:                  210
Model:             Multinomial Logit Model   Df Residuals:                      203
Method:                                MLE   Df Model:                            7
Date:                     Fri, 07 Feb 2020   Pseudo R-squ.:                   0.394
Time:                             17:03:01   Pseudo R-bar-squ.:               0.370
AIC:                               366.641   Log-Likelihood:               -176.320
BIC:                               390.070   LL-Null:                      -291.122
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
ASC_air            5.6860      0.937      6.068      0.000       3.849       7.523
ASC_train          5.4034      0.603      8.959      0.000       4.221       6.585
ASC_bus            5.0128      0.623      8.051      0.000       3.792       6.233
TTME              -0.0992      0.011     -9.428      0.000      -0.120      -0.079
INVT              -0.0039      0.001     -4.489      0.000      -0.006      -0.002
HINC_train_bus    -0.0500      0.011     -4.484      0.000      -0.072      -0.028
PSIZE_air         -0.8997      0.245     -3.680      0.000      -1.379      -0.420
==================================================================================
"""

In [8]:
# 创建用于预测的df
prediction_df = model_data[['OBS_ID', 'ALT_ID', 'MODE','TTME', 'INVT','HINC','PSIZE']]
choice_column = "MODE"
# 对火车耗时进行变化
def INVT(x,y):
    if x == 1:
        return y*0.8
    else:
        return y
prediction_df['INVT'] = prediction_df.apply(lambda x: INVT(x.ALT_ID, x.INVT), axis = 1)
# 默认情况下，predict方法返回每个选择情况下每个可用备选方案的预测概率。
prediction_array = mnl.predict(prediction_df)
# 存储预测概率
prediction_df["MNL_Predictions"] = prediction_array
# 对比变化前后的概率
raw_probability = prediction_df.groupby(['ALT_ID'])['MODE'].mean()
new_probability = prediction_df.groupby(['ALT_ID'])['MNL_Predictions'].mean()
print("--------原概率--------")
print(raw_probability)
print("--------新概率--------")
print(new_probability)

# | --------原概率--------
# | ALT_ID
# | 0    0.276190
# | 1    0.300000
# | 2    0.142857
# | 3    0.280952
# | Name: MODE, dtype: float64
# | --------新概率--------
# | ALT_ID
# | 0    0.255643
# | 1    0.362788
# | 2    0.126937
# | 3    0.254632

C:\Users\TBBack\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\TBBack\Anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:381: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe["intercept"] = 1.0
C:\Users\TBBack\Anaconda3\lib\site-packages\pylogit\choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is 

--------原概率--------
ALT_ID
0    0.276190
1    0.300000
2    0.142857
3    0.280952
Name: MODE, dtype: float64
--------新概率--------
ALT_ID
0    0.255643
1    0.362788
2    0.126937
3    0.254632
Name: MNL_Predictions, dtype: float64


In [9]:
# 创建用于预测的df
prediction_df = model_data[['OBS_ID', 'ALT_ID', 'MODE','TTME', 'INVT','HINC','PSIZE']]
choice_column = "MODE"
# 对家庭收入进行变化
prediction_df['HINC'] = prediction_df['HINC']*1.2
# 默认情况下，predict方法返回每个选择情况下每个可用备选方案的预测概率。
prediction_array = mnl.predict(prediction_df)
# 存储预测概率
prediction_df["MNL_Predictions"] = prediction_array
# 对比变化前后的概率
raw_probability = prediction_df.groupby(['ALT_ID'])['MODE'].mean()
new_probability = prediction_df.groupby(['ALT_ID'])['MNL_Predictions'].mean()
print("--------原概率--------")
print(raw_probability)
print("--------新概率--------")
print(new_probability)

C:\Users\TBBack\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


--------原概率--------
ALT_ID
0    0.276190
1    0.300000
2    0.142857
3    0.280952
Name: MODE, dtype: float64
--------新概率--------
ALT_ID
0    0.291839
1    0.271424
2    0.128988
3    0.307749
Name: MNL_Predictions, dtype: float64
